# マッチング推定

selection on observableの仮定

1. $(Y(0), Y(1)) \operatorname{\perp\!\!\perp} D \mid X$ (条件付き独立の仮定)
2. $0 < P(D=1 \mid X) < 1$（common support）

これにより、

$$
\begin{aligned}
E[Y(1)-Y(0) \mid X]
& =E[Y(1)-Y(0) \mid X, D=1] \\
& =E[Y(1) \mid X, D=1]-E[Y(0) \mid X, D=0] \\
& =E[Y \mid X, D=1]-E[Y \mid X, D=0]
\end{aligned}
$$

であり、この条件付き期待値をもとに$X$について期待値をとることでATEが推定できる

$$
ATE = E_X[E[Y(1)-Y(0) \mid X]]
$$

層別化（subclassification）は処置群・対照群の共変量の分布を同じにする（balanceさせる）こと

## Exact Matching

共変量$X$が完全に一致するペア同士で比較する

## Coarsend Exact Matching

共変量をグルーピングする、つまり粒度を荒く（coarsen）したあと、Exact Matchingを行う


結構有名で、Rのパッケージもあり、アメリカの医薬品の規制当局からも科学的な定量評価法として認可を受けてるらしい（[CEM: Coarsened Exact Matching Software | GARY KING](https://gking.harvard.edu/cem)）

- 論文: [Iacus, S. M., King, G., & Porro, G. (2012). Causal inference without balance checking: Coarsened exact matching. Political analysis, 20(1), 1-24.](https://gking.harvard.edu/files/political_analysis-2011-iacus-pan_mpr013.pdf)
